In [6]:
from pyincore import IncoreClient
from pyincore.analyses.populationdislocation import PopulationDislocation, PopulationDislocationUtil

In [7]:
# Connect to IN-CORE service
client = IncoreClient()

### Population dislocation for Joplin, MO

In [8]:
# Joplin, MO, Building damage, Joplin_bldg_dmg_result
building_dmg = "5dbc9525b9219c06dd282637"

# Joplin, MO, House unit allocation, 2ev2_housingunitallocation_1238
housing_unit_alloc = "5dc1c196b9219c06dd2e3f0b"

# Joplin, MO, Background data, 2ev2_2019-08-01_bgdata
bg_data = "5d4c9545b9219c0689b2358a"

# Value loss parameters, value_loss_bai09.csv
value_loss = "5dfd1069fc33d500081555d8"

In [9]:
# Create population dislocatin 
pop_dis = PopulationDislocation(client)

# Load input dataset
pop_dis.load_remote_input_dataset("building_dmg", building_dmg)
pop_dis.load_remote_input_dataset("housing_unit_allocation", housing_unit_alloc)
pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_poss_param", value_loss)

# Specify the result name
result_name = "IN-CORE_1bv6_population_dislocation"

seed = 1111

# Set analysis parameters
pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)

Dataset already exists locally. Reading from local cache.
Dataset already exists locally. Reading from local cache.
Dataset already exists locally. Reading from local cache.
Dataset already exists locally. Reading from local cache.


True

In [10]:
# Run Population dislocation analysis
pop_dis.run_analysis()

True

In [11]:
# Retrieve result dataset
result = pop_dis.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
df.head()

,guid,insignific,moderate,heavy,complete,strctid,statefp10,tractce10,pumgeoid10,pumname10,...,bgyear,pblackbg,phispbg,d_sf,rploss_ins,rploss_med,rploss_hwy,rploss_cmp,prdis,dislocated
0,b497b88f-a4ba-48b1-8438-5cfc68147cfc,1.0,0.0,0.0,0.0,S0018614,29.0,11100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,2.235294,4.470588,0,0.002914,0.100425,0.524702,0.889881,0.374637,True
1,6eadcc3e-7bb5-463a-aca8-f1d16f8ac7cc,1.0,0.0,0.0,0.0,S0018615,29.0,11100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,2.235294,4.470588,0,0.006930,0.132025,0.677548,0.951019,0.376972,False
2,758fb3de-108a-4e04-895e-fdc4b8627876,1.0,0.0,0.0,0.0,S0018616,29.0,11100.0,2902800.0,Jasper & Newton Counties PUMA,...,NaN,NaN,NaN,0,0.005776,0.100000,0.654935,0.941974,NaN,False
3,7b706697-bf9b-4883-90f6-862d67a8e64b,1.0,0.0,0.0,0.0,S0018617,29.0,11100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,2.235294,4.470588,0,0.002404,0.105851,0.436479,0.854591,0.374340,True
4,19c91f9b-f679-4263-b7a3-e76fd859c4b1,1.0,0.0,0.0,0.0,S0018618,29.0,11100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,2.235294,4.470588,0,0.003029,0.133077,0.438777,0.855511,0.374703,True
